You may also want to look at this after the tutorial in this file:
https://nbviewer.org/github/ysBach/IshiguroM_etal_155140_2005UD/blob/master/photometry/Photometer.ipynb

(Appendix of Ishiguro, Bach et al. (2022) MNRAS, 509, 4128 "Polarimetric properties of the near-Sun asteroid 155140 2005 UD in comparison with other asteroids and meteoritic samples")

In [2]:
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr' 
import numpy as np
from matplotlib import pyplot as plt

# We need to do it in a separate cell. See:
# https://github.com/jupyter/notebook/issues/3385
plt.style.use('default')
plt.rcParams.update({
    'font.family': 'latin modern math', 'font.size':12, 'mathtext.fontset':'stix', 
    'axes.formatter.use_mathtext': True, 'axes.formatter.limits': (-4, 4),
    'axes.grid': True, 'grid.color': 'gray', 'grid.linewidth': 0.5, 
    'xtick.top': True, 'ytick.right': True, 
    'xtick.direction': 'inout', 'ytick.direction': 'inout',
    'xtick.minor.size': 4.0, 'ytick.minor.size': 4.0,  # default 2.0
    'xtick.major.size': 8.0, 'ytick.major.size': 8.0,  # default 3.5
    'xtick.minor.visible': True, 'ytick.minor.visible': True
})
from astropy.time import Time
from astropy.table import Table, vstack
import ysfitsutilpy as yfu
import ysphotutilpy as ypu
import ysvisutilpy as yvu
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import units as u
from photutils.aperture import CircularAnnulus, CircularAperture
import numpy as np
import pandas as pd
from astropy.nddata import CCDData

#####################################################################
# Our object (will be queried to JPL HORIZONS)
OBJID = '216' # Kleopatra

# Observed location
LOCATION = dict(lon=127.0, lat=37.3, elevation=130)

# It is used as a rough estimate, so no need to be accurate:
PIX2ARCSEC = 1.24*u.arcsec

# Used for any `astropy.SkyCoord` object:
SKYC_KW = dict(unit=u.deg, frame='icrs')

# Initial guess of FWHM in pixel
FWHM_INIT = 6

# Photometry parameters
R_AP = 1.5*FWHM_INIT # Aperture radius
R_IN = 4*FWHM_INIT   # Inner radius of annulus
R_OUT = 6*FWHM_INIT  # Outer radius of annulus
#####################################################################

# Load the FITS file

In [3]:
ccd = yfu.load_ccd("KLEOPATRA_Light_r_2022-11-02-12-13-51_100sec_RiLA600_STX-16803_-20C_2bin_wcs_c.fit", unit="adu")

FileNotFoundError: [Errno 2] No such file or directory: 'KLEOPATRA_Light_r_2022-11-02-12-13-51_100sec_RiLA600_STX-16803_-20C_2bin_wcs_c.fit'

# Get the Ephemeris
Using [JPL HORIZONS](https://ssd.jpl.nasa.gov/horizons.cgi#top), implemented to [`astroquery.jplhorizons`](https://astroquery.readthedocs.io/en/latest/jplhorizons/jplhorizons.html)

In [ ]:
_, eph, _ = ypu.horizons_query(OBJID, epochs=Time(ccd.header["DATE-OBS"]).jd, location=LOCATION)
eph

In [ ]:
pos_targ_init = SkyCoord(eph["RA"], eph["DEC"], **SKYC_KW).to_pixel(ccd.wcs)
ap = CircularAperture([pos_targ_init[0][0], pos_targ_init[1][0]], r=R_AP)
an = CircularAnnulus([pos_targ_init[0][0], pos_targ_init[1][0]], r_in=R_IN, r_out=R_OUT)

pos_targ_init

# Initial Photometry of the Target

In [ ]:
phot_targ = ypu.apphot_annulus(ccd, ap, an, error=yfu.errormap(ccd))
phot_targ

# Query PS1 Objects in the FOV & Photometry

I used [`astroquery.vizier`](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html). However, note that the most modern catalog is available from MAST, e.g., [`astroquery.mast`](https://astroquery.readthedocs.io/en/latest/mast/mast.html).

In [ ]:
r_fov = yfu.fov_radius(ccd.header+ccd.wcs.to_header())
print(r_fov)
ps1 = ypu.PanSTARRS1(ccd.wcs.wcs.crval[0]*u.deg, ccd.wcs.wcs.crval[1]*u.deg, radius=r_fov,
                     column_filters={"rmag":"10.0..14.5", "e_rmag":"<0.10", "nr":">5"})
isnear = ypu.organize_ps1_and_isnear(
    ps1, 
    header=ccd.header+ccd.wcs.to_header(), 
    bezel=5*FWHM_INIT*PIX2ARCSEC.value,
    nearby_obj_minsep=5*FWHM_INIT*PIX2ARCSEC.value,
    group_crit_separation=6*FWHM_INIT
)
df_stars = ps1.queried.to_pandas()

In [ ]:
df_stars

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 5), sharex=False, sharey=False, gridspec_kw=None)

yvu.norm_imshow(axs, ccd, zscale=True)
ap = CircularAperture([pos_targ_init[0][0], pos_targ_init[1][0]], r=R_AP)
an = CircularAnnulus([pos_targ_init[0][0], pos_targ_init[1][0]], r_in=R_IN, r_out=R_OUT)
ap.plot(axs, color="r")
an.plot(axs, color="b")

_phot_stars = []

for i, row in df_stars.iterrows():
    pos_star = SkyCoord(row["RAJ2000"], row["DEJ2000"], **SKYC_KW).to_pixel(ccd.wcs)
    ap = CircularAperture([pos_star[0], pos_star[1]], r=R_AP)
    an = CircularAnnulus([pos_star[0], pos_star[1]], r_in=R_IN, r_out=R_OUT)
    _phot_star = ypu.apphot_annulus(ccd, ap, an, error=yfu.errormap(ccd))
    _phot_star["Rmag"] = row["Rmag"]
    _phot_star["e_Rmag"] = row["e_Rmag"]
    _phot_star["grcolor"] = row["grcolor"]
    _phot_star["e_grcolor"] = row["e_grcolor"]
    _phot_star["id"] = i
    _phot_star["objID"] = int(row["objID"])
    _phot_stars.append(_phot_star)
    axs.text(pos_star[0]+10, pos_star[1]+10, f"star {i}", fontsize=8)
    ap.plot(axs, color="orange")
    an.plot(axs, color="w")


plt.tight_layout()
plt.show();

In [ ]:
phot_stars = pd.concat(_phot_stars)
# phot_stars = phot_stars.loc[phot_stars["objID"] != 110823405221754720].copy()  # star 15
# SEE THE LAST CELL IN THIS FILE FOR DESCRIPTION
phot_stars

# Standardization Plots

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5), sharex=False, sharey=False, gridspec_kw=None)

_xx = np.linspace(13, 15)
axs.plot(phot_stars["Rmag"], phot_stars["mag"], '+')
axs.axhline(phot_targ["mag"].values, label="Kleopatra, instrumental mag")
axs.plot(_xx, _xx + np.median(phot_stars["mag"] - phot_stars["Rmag"]))

for _, row in phot_stars.iterrows():
    axs.text(row["Rmag"], row["mag"], int(row["id"]), fontsize=8)

axs.set(
    xlabel="R magnitude (PS1 to R_C filter by Tonry+2012)",
    ylabel="R_inst"
)
axs.legend()

plt.tight_layout()
plt.show();

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(9, 5), sharex=False, sharey=False, gridspec_kw=None)

axs[0].plot(phot_stars["Rmag"], phot_stars["mag"] - phot_stars["Rmag"], '+')
axs[1].plot(phot_stars["grcolor"], phot_stars["mag"] - phot_stars["Rmag"], '+')
for _, row in phot_stars.iterrows():
    axs[0].text(row["Rmag"], row["mag"] - row["Rmag"], int(row["id"]), fontsize=8)
    axs[1].text(row["grcolor"], row["mag"] - row["Rmag"], int(row["id"]), fontsize=8)
    
axs[0].set(
    xlabel="R magnitude (PS1 to R_C filter by Tonry+2012)",
    ylabel="R_inst - R"
)
axs[1].set(
    xlabel="g - r (PS1)",
    ylabel="R_inst - R"
)

plt.tight_layout()
plt.show();

In [ ]:
phot_targ

# What's Wrong with Star #15 ??

In [ ]:
print(f'{int(df_stars.iloc[15]["f_objID"]):031b}')

Maybe it is just a peculiar star (happen to be nearby another object) that we should remove...

**You can do it by uncommenting the commented line in cell 9**

The meanings of binary flags can be found at [here](https://outerspace.stsci.edu/display/PANSTARRS/PS1+Object+Flags).

Below is a simple description of this star 15:
```
'0011110000001001110000000000000'
#  ^^^^      ^  ^^^  
#3         2         1
#0987654321098765432109876543210
# flagged at 13, 14, 15, 18, 25, 26, 27, 28
#         * FIT_AVE (8192) : average position was fitted
#         * FIT_PM (16384) : proper motion model was fitted
#         * FIT_PAR (32768) : parallax model was fitted
#         * USE_PAR (262144) : parallax used (not AVE or PM)
#         * GOOD (33554432) : good-quality measurement in our data (eg,PS)
#         * GOOD_ALT (67108864) : good-quality measurement in external data
#         (eg, 2MASS)
#         * GOOD_STACK (134217728) : good-quality object in the stack (>1 good
#         stack measurement)
#         * BEST_STACK (268435456) : the primary stack measurements are the
#         best measurements
```